In [1]:
# %pip install 'accelerate>=0.26.0'
# %pip install torch
# %pip install transformers
# %pip install accelerate
# %pip install -U transformers bitsandbytes
# !python -m pip install optimum


In [3]:
import torch
from transformers import pipeline
import json

# Initialize pipeline
pipe = pipeline(
    task="text-generation",
    model="Qwen/Qwen2.5-1.5B-Instruct",
    max_new_tokens=512
)

def save_conversation(chat_history, filename="cooking_session.json"):
    with open(filename, 'w') as f:
        json.dump(chat_history, f, indent=2)
    print(f"Saved to {filename}")

def cook_recipe(recipe_data, automated_inputs):
    # Initialize fresh conversation
    chat = [
        {
            "role": "system", 
            "content": "You are a master chef. Guide through recipe steps sequentially. Start with step 1 after recipe is provided."
        }
    ]
    
    # Structured recipe submission
    chat.append({
        "role": "user",
        "content": f"""Guide me through this recipe step-by-step:
        
        TITLE: {recipe_data['title']}
        
        INGREDIENTS:
        {chr(10).join(recipe_data['ingredients'])}
        
        INSTRUCTIONS:
        {recipe_data['instructions']}"""
    })
    
    # Process automated inputs
    for user_input in automated_inputs:
        # Generate response
        outputs = pipe(chat)
        chef_response = outputs[0]["generated_text"][-1]["content"]
        print(f"\nChef: {chef_response}")
        chat.append({"role": "assistant", "content": chef_response})

        # Add automated input
        print(f"\nTrainee: {user_input}")
        chat.append({"role": "user", "content": user_input})

        if "exit" in user_input.lower():
            break
    
    save_conversation(chat)
    return chat

# Recipe data structure
recipe = {
    "title": "Miso-Butter Roast Chicken With Acorn Squash Panzanella",
    "ingredients": [
        '1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more',
        '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage',
        '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature',
        '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper',
        '⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)',
        '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces',
        '2 Tbsp. extra-virgin olive oil', '½ small red onion, thinly sliced',
        '3 Tbsp. apple cider vinegar', '1 Tbsp. white miso', '¼ cup all-purpose flour',
        '2 Tbsp. unsalted butter, room temperature', '¼ cup dry white wine',
        '2 cups unsalted chicken broth', '2 tsp. white miso', 'Kosher salt, freshly ground pepper'
    ],
    "instructions": """
    1. Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine.
    2. Halve squash and scoop out seeds...
    (Include full instructions here)
    """
}

# Predefined automated trainee inputs
automated_inputs = [
    "Got it! Ready for step 1.",
    "Done with step 1. Continue.",
    # "Finished step 2! What's next?",
    # "I need clarification on peeling the squash.",
    # "Okay to proceed.",
    "exit"
]

# Start cooking session with automated inputs
cook_recipe(recipe, automated_inputs)


Device set to use mps:0



Chef: Certainly! Let's guide you through the detailed steps for making Miso-Butter Roast Chicken With Acorn Squash Panzanella:

### Step 1: Prepare the Ingredients

First, let's gather all the necessary ingredients to ensure we have everything ready before starting the cooking process.

**Ingredients:**
- **Chicken:** 1 (3½–4-lb.) whole chicken
- **Acorn Squash:** 2 small acorn squash (about 3 lb. total)
- **Sage:** 2 Tbsp. finely chopped
- **Rosemary:** 1 Tbsp. finely chopped
- **Melted Butter:** 6 Tbsp. unsalted butter, plus 3 Tbsp. room temperature
- **Allspice:** ¼ tsp. ground allspice
- **Red Pepper Flakes:** Pinch of crushed red pepper flakes
- **Black Pepper:** Freshly ground black pepper
- **White Bread:** ⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)
- **Apples:** 2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces
- **Extra-Virgin Olive Oil:** 2 Tbsp.
- **Red Onion:** 1 small red onion, thinly sliced
- **Ap

[{'role': 'system',
  'content': 'You are a master chef. Guide through recipe steps sequentially. Start with step 1 after recipe is provided.'},
 {'role': 'user',
  'content': 'Guide me through this recipe step-by-step:\n        \n        TITLE: Miso-Butter Roast Chicken With Acorn Squash Panzanella\n        \n        INGREDIENTS:\n        1 (3½–4-lb.) whole chicken\n2¾ tsp. kosher salt, divided, plus more\n2 small acorn squash (about 3 lb. total)\n2 Tbsp. finely chopped sage\n1 Tbsp. finely chopped rosemary\n6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature\n¼ tsp. ground allspice\nPinch of crushed red pepper flakes\nFreshly ground black pepper\n⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)\n2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces\n2 Tbsp. extra-virgin olive oil\n½ small red onion, thinly sliced\n3 Tbsp. apple cider vinegar\n1 Tbsp. white miso\n¼ cup all-purpose flour\n2 Tbsp. unsalted butter